# E-Commerce with Virtual Mouse

In [9]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np
import math
import time
import webbrowser
import threading
from flask import Flask, request
import json
import os

Configuration

In [10]:
WEBCAM_ID = 0
CAM_WIDTH = 700
CAM_HEIGHT = 540
SMOOTHING = 5
FRAME_REDUCTION = 100
SERVER_PORT = 5000

Global Shutdown Event for Threading

In [11]:
shutdown_event = threading.Event()

Product Data

In [12]:
PRODUCTS_DB = [
    { 'id': 'p1', 'name': 'Apple Watch Series 10', 'price': 42999, 'image': 'product_images/apple-watch-series-10.jpg' },
    { 'id': 'p2', 'name': 'Meta Quest 3 (128GB)', 'price': 51990, 'image': 'product_images/meta-quest-3.jpg' },
    { 'id': 'p3', 'name': 'Apple iPhone 16 Pro Max', 'price': 134900, 'image': 'product_images/iphone-16-pro-max.jpg' },
    { 'id': 'p4', 'name': 'Samsung Galaxy S25 Ultra', 'price': 129999, 'image': 'product_images/samsung-s25-ultra.jpg' },
    { 'id': 'p5', 'name': 'OnePlus 13 (12GB RAM, 256GB)', 'price': 64999, 'image': 'product_images/oneplus-13.jpg' },
    { 'id': 'p6', 'name': 'Sony WH-1000XM5 Headphones', 'price': 29990, 'image': 'product_images/sony-wh-1000xm5-headphones.jpg' },
    { 'id': 'p7', 'name': 'Bose QuietComfort Ultra', 'price': 35900, 'image': 'product_images/bose-quiet-comfort-ultra.jpg' },
    { 'id': 'p8', 'name': 'DJI Mini 4 Pro Drone', 'price': 99000, 'image': 'product_images/dji-mini-4-pro-drone.jpg' },
    { 'id': 'p9', 'name': 'GoPro HERO12 Black', 'price': 39990, 'image': 'product_images/go-pro-hero-12.jpg' },
    { 'id': 'p10', 'name': 'Logitech MX Master 3S', 'price': 9495, 'image': 'product_images/logitech-mx-master-3s.jpg' },
    { 'id': 'p11', 'name': 'Razer BlackWidow V4 Pro', 'price': 22999, 'image': 'product_images/razer-black-widow-v4-pro.jpg' },
    { 'id': 'p12', 'name': 'Samsung Odyssey OLED G9', 'price': 175000, 'image': 'product_images/samsung-odyssey-oled-g9.jpg' },
    { 'id': 'p13', 'name': 'LG C3 55" OLED TV', 'price': 119990, 'image': 'product_images/lg-c3-oled-tv.jpg' },
    { 'id': 'p14', 'name': 'Apple MacBook Air (M3)', 'price': 114900, 'image': 'product_images/apple-macbook-air-m3.jpg' },
    { 'id': 'p15', 'name': 'Apple iPad Pro (M4)', 'price': 169900, 'image': 'product_images/apple-ipad-pro-m4.jpg' },
    { 'id': 'p16', 'name': 'Samsung Galaxy Tab S9', 'price': 72999, 'image': 'product_images/samsung-galaxy-tab-s9.jpg' },
    { 'id': 'p17', 'name': 'Amazon Kindle Paperwhite', 'price': 16829, 'image': 'product_images/amazon-kindle-paper-white.jpg' },
    { 'id': 'p18', 'name': 'Dyson V15 Detect Vacuum', 'price': 83693, 'image': 'product_images/dyson-v15-detect-vaccum.jpg' },
    { 'id': 'p19', 'name': 'Anker 737 Power Bank', 'price': 9999, 'image': 'product_images/anker-737-power-bank.jpg' },
    { 'id': 'p20', 'name': 'Belkin 3-in-1 Charger', 'price': 9999, 'image': 'product_images/belkin-3-in-1-charger.jpg' },
    { 'id': 'p21', 'name': 'Sony PlayStation 5', 'price': 61039, 'image': 'product_images/sany-playstation-5.jpg' },
    { 'id': 'p22', 'name': 'Nintendo Switch OLED', 'price': 28999, 'image': 'product_images/nintendo-switch-oled.jpg' }
]

HTML Frontend

In [13]:
def get_html_content():
    product_cards_html = ""
    for product in PRODUCTS_DB:
        product_cards_html += f"""
        <div class="bg-white rounded-lg shadow-md p-4 flex flex-col product-card">
            <div class="w-full aspect-square rounded-md mb-4 overflow-hidden">
                <img src="{product['image']}" alt="{product['name']}" class="w-full h-full object-contain">
            </div>
            <h2 class="text-lg font-bold mb-1 flex-grow mt-2">{product['name']}</h2>
            <div class="flex justify-between items-center mt-4">
                <span class="text-xl font-bold text-indigo-600">₹{product['price']:,}</span>
                <button data-id="{product['id']}" class="add-to-cart-btn bg-indigo-500 text-white font-bold py-2 px-3 rounded-lg hover:bg-indigo-600 transition-colors text-sm">Add to Cart</button>
            </div>
        </div>
        """

    return f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>GestureMart - Python Edition</title>
        <script src="https://cdn.tailwindcss.com"></script>
        <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">
        <style>
            body {{ font-family: 'Inter', sans-serif; }}
            .product-card:hover {{ transform: translateY(-5px); box-shadow: 0 10px 15px -3px rgb(0 0 0 / 0.1), 0 4px 6px -4px rgb(0 0 0 / 0.1); }}
            .product-card {{ transition: transform 0.3s ease, box-shadow 0.3s ease; }}
        </style>
    </head>
    <body class="bg-gray-100 text-gray-800">
        <div class="container mx-auto px-4 py-8">
            <header class="flex justify-between items-center mb-8 pb-4 border-b border-gray-300">
                <h1 class="text-4xl font-bold text-indigo-600">GestureMart</h1>
                <div class="flex items-center space-x-4">
                    <div class="text-lg"><span class="font-semibold">Cart:</span> <span id="cart-item-count">0</span> items</div>
                    <button id="view-cart-btn" class="bg-indigo-500 text-white font-bold py-2 px-4 rounded-lg hover:bg-indigo-600 transition-colors">View Cart</button>
                </div>
            </header>
            <div id="products-grid" class="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-4 gap-6">
                {product_cards_html}
            </div>
        </div>
        <div id="cart-modal" class="hidden fixed inset-0 bg-gray-900 bg-opacity-75 flex items-center justify-center z-50">
            <div class="bg-white rounded-lg shadow-2xl w-full max-w-3xl max-h-[90vh] flex flex-col">
                <div class="flex justify-between items-center p-6 border-b"><h2 class="text-2xl font-bold">Your Shopping Cart</h2><button id="close-cart-btn" class="text-gray-500 hover:text-gray-800 text-2xl">&times;</button></div>
                <div id="cart-items-container" class="p-6 overflow-y-auto"></div>
                <div class="p-6 border-t mt-auto">
                    <div class="space-y-2 text-lg">
                        <div class="flex justify-between"><span>Subtotal:</span> <span id="cart-subtotal">₹0.00</span></div>
                        <div class="flex justify-between"><span>GST (18%):</span> <span id="cart-gst">₹0.00</span></div>
                        <div class="flex justify-between font-bold text-xl"><span>Total Payable:</span> <span id="cart-total">₹0.00</span></div>
                    </div>
                    <button id="checkout-btn" class="bg-emerald-500 text-white font-bold py-3 px-6 rounded-lg text-xl w-full mt-6 hover:bg-emerald-600 transition-colors">Checkout</button>
                </div>
            </div>
        </div>
        <div id="success-modal" class="hidden fixed inset-0 bg-gray-900 bg-opacity-75 flex items-center justify-center z-50">
            <div class="bg-white p-8 rounded-lg shadow-2xl text-center">
                <h2 class="text-3xl font-bold text-emerald-500 mb-4">Thank You!</h2>
                <p class="text-lg text-gray-700">Items Bought Successfully!</p>
                <p class="text-sm text-gray-500 mt-2">Application will close in <span id="countdown">10</span> seconds.</p>
            </div>
        </div>
        <script>
            const productsData = {json.dumps(PRODUCTS_DB)};
            let cart = [];
            const cartItemCountEl = document.getElementById('cart-item-count');
            const viewCartBtn = document.getElementById('view-cart-btn');
            const closeCartBtn = document.getElementById('close-cart-btn');
            const cartModal = document.getElementById('cart-modal');
            const checkoutBtn = document.getElementById('checkout-btn');
            function findProduct(productId) {{ return productsData.find(p => p.id === productId); }}
            function updateCart() {{
                const totalItems = cart.reduce((sum, item) => sum + item.quantity, 0);
                cartItemCountEl.textContent = totalItems;
                const cartItemsContainer = document.getElementById('cart-items-container');
                cartItemsContainer.innerHTML = '';
                if (cart.length === 0) {{
                    cartItemsContainer.innerHTML = '<p class="text-gray-500">Your cart is empty.</p>';
                }} else {{
                    cart.forEach(item => {{
                        const cartItemEl = document.createElement('div');
                        cartItemEl.className = 'flex items-center justify-between py-3 border-b';
                        cartItemEl.innerHTML = `
                            <div class="w-3/5"><p class="font-semibold">${{item.name}}</p><p class="text-sm text-gray-600">₹${{item.price.toLocaleString('en-IN')}}</p></div>
                            <div class="flex items-center space-x-3">
                                <button data-id="${{item.id}}" class="decrease-qty-btn font-bold w-6 h-6 bg-gray-200 rounded-full">-</button>
                                <span>${{item.quantity}}</span>
                                <button data-id="${{item.id}}" class="increase-qty-btn font-bold w-6 h-6 bg-gray-200 rounded-full">+</button>
                            </div>
                            <button data-id="${{item.id}}" class="remove-item-btn text-red-500 hover:text-red-700 font-semibold">Remove</button>
                        `;
                        cartItemsContainer.appendChild(cartItemEl);
                    }});
                }}
                const subtotal = cart.reduce((sum, item) => sum + item.price * item.quantity, 0);
                const gst = subtotal * 0.18;
                const total = subtotal + gst;
                document.getElementById('cart-subtotal').textContent = `₹${{subtotal.toLocaleString('en-IN', {{ minimumFractionDigits: 2 }})}}`;
                document.getElementById('cart-gst').textContent = `₹${{gst.toLocaleString('en-IN', {{ minimumFractionDigits: 2 }})}}`;
                document.getElementById('cart-total').textContent = `₹${{total.toLocaleString('en-IN', {{ minimumFractionDigits: 2 }})}}`;
            }}
            document.getElementById('products-grid').addEventListener('click', e => {{
                if (e.target.classList.contains('add-to-cart-btn')) {{
                    const productId = e.target.dataset.id;
                    const existingItem = cart.find(item => item.id === productId);
                    if (existingItem) {{ existingItem.quantity++; }}
                    else {{ cart.push({{ ...findProduct(productId), quantity: 1 }}); }}
                    updateCart();
                }}
            }});
            document.getElementById('cart-items-container').addEventListener('click', e => {{
                const productId = e.target.dataset.id;
                if (!productId) return;
                const item = cart.find(i => i.id === productId);
                if (e.target.classList.contains('increase-qty-btn')) {{ item.quantity++; }}
                if (e.target.classList.contains('decrease-qty-btn')) {{
                    item.quantity--;
                    if (item.quantity === 0) {{ cart = cart.filter(i => i.id !== productId); }}
                }}
                if (e.target.classList.contains('remove-item-btn')) {{ cart = cart.filter(i => i.id !== productId); }}
                updateCart();
            }});
            viewCartBtn.addEventListener('click', () => cartModal.classList.remove('hidden'));
            closeCartBtn.addEventListener('click', () => cartModal.classList.add('hidden'));
            checkoutBtn.addEventListener('click', () => {{
                if (cart.length > 0) {{
                    cartModal.classList.add('hidden');
                    document.getElementById('success-modal').classList.remove('hidden');
                    let seconds = 10;
                    const interval = setInterval(() => {{
                        seconds--;
                        document.getElementById('countdown').textContent = seconds;
                        if (seconds <= 0) {{
                            clearInterval(interval);
                            fetch('/shutdown').then(() => {{
                                document.body.innerHTML = '<div class="w-screen h-screen flex items-center justify-center bg-gray-900 text-white text-2xl">Session Ended. Closing...</div>';
                            }});
                        }}
                    }}, 1000);
                }} else {{ alert("Your cart is empty!"); }}
            }});
        </script>
    </body>
    </html>
    """

Flask Web Server

In [14]:
app = Flask(__name__, static_folder='product_images')


@app.route('/')
def home():
    return get_html_content()


@app.route('/shutdown')
def shutdown():
    shutdown_event.set()
    func = request.environ.get('werkzeug.server.shutdown')
    if func: func()
    return 'Server shutting down...'


def run_flask_app():
    if not os.path.exists('product_images'):
        print("Error: 'product_images' folder not found. Please create it next to the script.")
        print("Application cannot start without the image folder.")
        shutdown_event.set()
        return
    app.run(port=SERVER_PORT, debug=False, use_reloader=False)

Virtual Mouse Functionality

In [15]:
def run_virtual_mouse():
    p_time = 0
    p_loc_x, p_loc_y = 0, 0
    c_loc_x, c_loc_y = 0, 0

    # --- FIX: Re-engineered scrolling state variables ---
    is_scrolling = False
    scroll_anchor_y = 0

    cap = cv2.VideoCapture(WEBCAM_ID)
    if not cap.isOpened():
        print(f"Error: Could not open webcam with ID {WEBCAM_ID}.")
        return
    cap.set(3, CAM_WIDTH)
    cap.set(4, CAM_HEIGHT)
    screen_width, screen_height = pyautogui.size()
    pyautogui.FAILSAFE = False
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.5)
    mp_draw = mp.solutions.drawing_utils
    print("AI Virtual Mouse thread started.")

    while not shutdown_event.is_set():
        success, frame = cap.read()
        if not success: continue
        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]

            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
            ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
            index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
            middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
            ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]

            h, w, c = frame.shape

            three_fingers_up = (index_tip.y < index_mcp.y and
                                middle_tip.y < middle_mcp.y and
                                ring_tip.y < ring_mcp.y)

            if three_fingers_up:
                # SCROLLING MODE
                if not is_scrolling:
                    is_scrolling = True
                    scroll_anchor_y = index_tip.y # Set anchor point when gesture starts

                # Calculate displacement from the anchor
                displacement = index_tip.y - scroll_anchor_y

                # Implement a dead zone to prevent jitter
                dead_zone = 0.03
                if abs(displacement) > dead_zone:
                    # Calculate scroll amount based on how far the hand is from the anchor
                    scroll_velocity = (displacement - (dead_zone * np.sign(displacement))) * -250
                    pyautogui.scroll(int(scroll_velocity))

                cv2.putText(frame, "SCROLLING", (w // 2 - 100, 50), cv2.FONT_HERSHEY_PLAIN, 3, (0, 165, 255), 3)

            else:
                # CURSOR & CLICK MODE
                is_scrolling = False # Reset scroll mode when gesture ends

                ix, iy = int(index_tip.x * w), int(index_tip.y * h)
                screen_x = np.interp(ix, (FRAME_REDUCTION, w - FRAME_REDUCTION), (0, screen_width))
                screen_y = np.interp(iy, (FRAME_REDUCTION, h - FRAME_REDUCTION), (0, screen_height))
                c_loc_x = p_loc_x + (screen_x - p_loc_x) / SMOOTHING
                c_loc_y = p_loc_y + (screen_y - p_loc_y) / SMOOTHING
                pyautogui.moveTo(c_loc_x, c_loc_y)
                p_loc_x, p_loc_y = c_loc_x, c_loc_y

                if math.hypot(middle_tip.x - index_tip.x, middle_tip.y - index_tip.y) < 0.05:
                    pyautogui.click()
                    cv2.circle(frame, (ix, iy), 15, (0, 255, 0), cv2.FILLED)
                    time.sleep(0.2)

            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        c_time = time.time()
        fps = 1 / (c_time - p_time) if (c_time - p_time) > 0 else 0
        p_time = c_time
        cv2.putText(frame, f'FPS: {int(fps)}', (20, 50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)
        cv2.imshow("AI Virtual Mouse - Press 'q' in this window to force quit", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            shutdown_event.set()
            break

    print("Virtual mouse thread shutting down...")
    cap.release()
    cv2.destroyAllWindows()
    hands.close()

Main Execution Block

In [16]:
if __name__ == '__main__':
    print("Starting application...")
    flask_thread = threading.Thread(target=run_flask_app, daemon=True)
    flask_thread.start()

    time.sleep(1)
    if shutdown_event.is_set():
        print("Exiting due to setup error.")
    else:
        print(f"Web server started on http://127.0.0.1:{SERVER_PORT}")
        mouse_thread = threading.Thread(target=run_virtual_mouse)
        mouse_thread.start()
        webbrowser.open_new(f"http://127.0.0.1:{SERVER_PORT}")
        print("Browser opened. The application is now running.")
        shutdown_event.wait()
        print("Main thread received shutdown signal. Terminating.")
        mouse_thread.join()
        print("Application has terminated successfully.")

Starting application...
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Web server started on http://127.0.0.1:5000
Browser opened. The application is now running.
AI Virtual Mouse thread started.
Main thread received shutdown signal. Terminating.
Virtual mouse thread shutting down...
Application has terminated successfully.
